In [20]:
# DataFest 2023: Mean / Mode
# Goal: Find the Mean (or Mode if more useful) of major Client, Question, and QuestionPost attributes 
    # ex) 11% of clients live in Texas, and 41% of questions are related to Family 
# Why: Help find relevant data that can help connect clients with attorneys
# Note: There's ~330k clients, so we uncomment info you don't need if we want it to load faster

import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


# Set up files
# Note: merged.parquet is Questions & QuestionPosts cleaned and merged together
# --------------------------------------------------------------------------------------------------------------------
# Read clients.csv and merged.parquet and merge them together into one dataframe
clients_df = pd.read_csv('csv_files/clients.csv')
merged_table = pq.read_table('csv_files/merged.parquet') 
merged_df = merged_table.to_pandas()

# Merge the DataFrames based on ClientUno and AskedByClientUno
# Note: These keys link the tables together so we can identify which client asked each question
combined_df = pd.merge(clients_df, merged_df, left_on='ClientUno', right_on='AskedByClientUno')


#Most Common Category: Family & Children at 40.77%
# --------------------------------------------------------------------------------------------------------------------
# Calculate the most common unique identifier in Category
most_common_category = merged_df['Category'].value_counts().idxmax()
# Calculate the percentage of how much the most common unique identifier takes up of combined_df
percentage = (merged_df['Category'].value_counts()[most_common_category] / len(merged_df)) * 100
print("Most common question Category: ", most_common_category)
print("Percentage of how many " + most_common_category + " questions there are: " + "{:.2f}%\n".format(percentage))


#Average NumberInHousehold: 1, 22% of the time 
# --------------------------------------------------------------------------------------------------------------------
combined_df['NumberInHousehold'] = clients_df['NumberInHousehold'].fillna(1)
most_common_number = clients_df['NumberInHousehold'].value_counts().idxmax()
percentage = (clients_df['NumberInHousehold'].value_counts()[most_common_number] / len(clients_df)) * 100
print("Most common NumberInHousehold: ", most_common_number)
print("Percentage of how many clients live a household of " + str(most_common_number) + " (assuming null counts as 1): {:.2f}% \n".format(percentage))


#Most Common Sub-Category: Family/Divorce/Custody at 17.32%
# --------------------------------------------------------------------------------------------------------------------
# Calculate the most common unique identifier in Subcategory
most_common_subcategory = merged_df['Subcategory'].value_counts().idxmax()
# Calculate the percentage of how much the most common unique identifier takes up of combined_df
percentage = (merged_df['Subcategory'].value_counts()[most_common_subcategory] / len(merged_df)) * 100
print("Most common question Subcategory", most_common_subcategory)
print("Percentage of how many " + most_common_subcategory + " questions there are: {:.2f}% \n".format(percentage))

#Most Common State: Texas at 11%
# --------------------------------------------------------------------------------------------------------------------
# Calculate the most common state name in StateName column 
most_common_state_name = clients_df['StateName'].value_counts().idxmax()
# Calculate the percentage of how much the most common state name takes up of combined_df
percentage = (clients_df['StateName'].value_counts()[most_common_state_name] / len(clients_df)) * 100
print("Most common state name: ", most_common_state_name)
print("Percentage of how many clients live in " + most_common_state_name + ": {:.2f}% \n".format(percentage))

#Average Annual Income: $41,800,579
# --------------------------------------------------------------------------------------------------------------------
# Calculate the average AnnualIncome of clients_df while ignoring null/0 values
# Calculate the median and IQR of the AnnualIncome column
median_income = clients_df['AnnualIncome'].median()
Q1 = clients_df['AnnualIncome'].quantile(0.25)
Q3 = clients_df['AnnualIncome'].quantile(0.75)
IQR = Q3 - Q1

# Calculate the upper bound for massive outliers
# --------------------------------------------------------------------------------------------------------------------
massive_outlier_bound = Q3 + 3.0 * IQR
# Filter out massive outliers and calculate the average income
filtered_df = clients_df[clients_df['AnnualIncome'] <= massive_outlier_bound]
average_income = filtered_df['AnnualIncome'].mean()
print("Average AnnualIncome of clients_df (ignoring massive outliers): ${:.2f} \n".format(average_income))

#Average Gender: Female,55%
# --------------------------------------------------------------------------------------------------------------------
#Replace null values in Gender column with "I'd rather not answer"
clients_df['Gender'] = clients_df['Gender'].fillna("I'd rather not answer")
# Calculate the count of each gender category
gender_counts = clients_df['Gender'].value_counts()
# Calculate the percentage of clients who are female
percentage_female = (gender_counts['Female'] / len(clients_df)) * 100
# Print the percentage of clients who are female to the console
print("Percentage of clients who are female: {:.2f}% \n".format(percentage_female))

#Average Age: 40
# --------------------------------------------------------------------------------------------------------------------
# Replace null values in Age column with the average age
average_age = clients_df['Age'].mean(skipna=True)
clients_df['Age'] = clients_df['Age'].fillna(average_age)
# Calculate the average age of clients_df
average_age = clients_df['Age'].mean()
print("Average age of clients {:.2f} \n".format(average_age))

Most common question Category:  Family and Children
Percentage of how many Family and Children questions there are: 40.77%

Most common NumberInHousehold:  1.0
Percentage of how many clients live a household of 1.0 (assuming null counts as 1): 22.72% 

Most common question Subcategory Family/Divorce/Custody
Percentage of how many Family/Divorce/Custody questions there are: 17.32% 

Most common state name:  Texas
Percentage of how many clients live in Texas: 10.88% 

Average AnnualIncome of clients_df (ignoring massive outliers): $25097.82 

Percentage of clients who are female: 55.95% 

Average age of clients 40.34 

